<a href="https://colab.research.google.com/github/Charliebond125/CE889_Group_Project/blob/main/Preprocessing_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns
import warnings

from math import sqrt

np.random.seed(42)

%matplotlib inline

warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def date_time(df):
    df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
    df["Month"] = df["Date"].dt.month
    df["Quarter"] = df["Date"].dt.quarter
    df["Year"] = df["Date"].dt.year
    df["Day"] = df["Date"].dt.day
    df["Week"] = df["Date"].dt.week
    
    df["Season"] = (
        np.where(df["Month"].isin([3,4,5]),
                "Spring", np.where(df["Month"].isin([6,7,8]),
                "Summer",np.where(df["Month"].isin([9,10,11]),
                    "Autumn",np.where(df["Month"].isin([12,1,2]),
                        "Winter","None"))))      
                        )
    return df

In [ ]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        ratio = len(df[col].value_counts()) / len(df)
        if ratio < 0.05:
            df[col] = df[col].astype('category')
    return df

In [ ]:
def one_hot(df):
    return pd.get_dummies(df)

In [ ]:
def calculate_outlier(df, column): ## function for calculating outliers
    Q3 = df[column].quantile(0.75)
    Q1 = df[column].quantile(0.25)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    percent_outliers = round(((df[df[column] > upper].shape[0]) + (df[df[column] < lower].shape[0])) / df.shape[0] * 100, 2)
    
    return lower, upper, percent_outliers

In [ ]:
# process train and test
def process_train_valid(data, isTest = False):
    # label encode some features
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    # extract some features from date column  
    data['Month'] = data.Date.dt.month
    data['Year'] = data.Date.dt.year
    data['Day'] = data.Date.dt.day
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    # calculate competiter open time in months
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    data['CompetitionOpen'] = data['CompetitionOpen'].apply(lambda x: x if x > 0 else 0)
    
    # calculate promo2 open time in months
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data['PromoOpen'].apply(lambda x: x if x > 0 else 0)
                                                 
    # Indicate whether the month is in promo interval
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['month_str'] = data.Month.map(month2str)

    def check(row):
        if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval']:
            return 1
        else:
            return 0
        
    data['IsPromoMonth'] =  data.apply(lambda row: check(row),axis=1)    
    
    # select the features we need
    features = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Day',
       'WeekOfYear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth']  
    if not isTest:
        features.append('Sales')
        
    data = data[features]
    return data


In [ ]:
def add_missing_columns( d, columns ):
        missing_cols = set( columns ) - set( d.columns )
        for c in missing_cols:
            d[c] = 0

def fix_columns( d, columns ):  

    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols: print ("extra columns:", extra_cols)

    d = d[ columns ]
    return d

In [ ]:
# Import data if using on local machine

df_train = pd.read_csv(r"C:\Users\charl\OneDrive\Desktop\rossmann-store-sales\train.csv")
df_test = pd.read_csv(r"C:\Users\charl\OneDrive\Desktop\rossmann-store-sales\test.csv")
df_store = pd.read_csv(r"C:\Users\charl\OneDrive\Desktop\rossmann-store-sales\store.csv")

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/test.csv")
df_store = pd.read_csv("/content/drive/MyDrive/store.csv")

In [ ]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [ ]:
df_store.fillna(0, inplace=True)

In [ ]:
df_test.fillna(0, inplace=True)

In [ ]:
train = pd.merge(df_train, df_store, on='Store')
test = pd.merge(df_test, df_store, on='Store')

# split the last 6 weeks data as hold-out set (idea from Gert https://www.kaggle.com/c/rossmann-store-sales/discussion/18024)
train = train.sort_values(['Date'],ascending = False)
train_total = train.copy()

split_index = 6*7*1115
valid = train[:split_index] 
train = train[split_index:]

# only use data of Sales>0 and Open is 1
valid = valid[(valid.Open != 0)&(valid.Sales >0)]
train = train[(train.Open != 0)&(train.Sales >0)]
train_total = train_total[(train_total.Open != 0)&(train_total.Sales >0)]

In [ ]:
# Ensuring outliers from sales are removed ## train

col_sales = 'Sales'
lower_sales, upper_sales, percent_outliers_sales = calculate_outlier(train, col_sales)

print("lower band = " + str(lower_sales))
print("upper band = " + str(upper_sales))
print("percentage of sales that are outliers = " + str(percent_outliers_sales) + "%")
train[train[col_sales] > upper_sales]
train[train[col_sales] < lower_sales]

train.loc[train[col_sales] > upper_sales, 'Sales'] = 14650
train.loc[train[col_sales] < lower_sales, 'Sales'] = -2366

print("Removing identified outliers for sales..." )

print("Completed: See table for confirmation. ")

train[train['Sales'] > 14650]
train[train['Sales'] < -2366]

lower band = -400.0
upper band = 13616.0
percentage of sales that are outliers = 3.64%
Removing identified outliers for sales...
Completed: See table for confirmation. 


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [ ]:
# customer outliers in train
col_customers = 'Customers'
lower_customers, upper_customers, percent_outliers_customers = calculate_outlier(train, col_customers)


print("higher band = " + str(upper_customers))

print("percentage of customers that are outliers = " + str(percent_outliers_customers) + "%")

train[train[col_customers] > upper_customers]
train.loc[train[col_customers] > upper_customers, 'Customers'] = 1485


print("Removing identified outliers for customers..." )
print("See table for confirmation. ")

train[train['Customers'] > 1485]


higher band = 1457.5
percentage of customers that are outliers = 4.84%
Removing identified outliers for customers...
See table for confirmation. 


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [ ]:
# removing validation outliers ## sales

col_sales = 'Sales'
lower_sales, upper_sales, percent_outliers_sales = calculate_outlier(valid, col_sales)

print("lower band = " + str(lower_sales))
print("upper band = " + str(upper_sales))
print("percentage of sales that are outliers = " + str(percent_outliers_sales) + "%")
valid[valid[col_sales] > upper_sales]
valid[valid[col_sales] < lower_sales]

valid.loc[valid[col_sales] > upper_sales, 'Sales'] = 14650
valid.loc[valid[col_sales] < lower_sales, 'Sales'] = -2366

print("Removing identified outliers for sales..." )

print("Completed: See table for confirmation. ")

valid[valid['Sales'] > 14650]
valid[valid['Sales'] < -2366]

lower band = -252.375
upper band = 13538.625
percentage of sales that are outliers = 3.6%
Removing identified outliers for sales...
Completed: See table for confirmation. 


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [ ]:
# removing outliers from customers ## validation

col_customers = 'Customers'
lower_customers, upper_customers, percent_outliers_customers = calculate_outlier(valid, col_customers)


print("higher band = " + str(upper_customers))

print("percentage of customers that are outliers = " + str(percent_outliers_customers) + "%")

valid[valid[col_customers] > upper_customers]
valid.loc[valid[col_customers] > upper_customers, 'Customers'] = 1485


print("Removing identified outliers for customers..." )
print("See table for confirmation. ")

valid[valid['Customers'] > 1485]

higher band = 1361.0
percentage of customers that are outliers = 4.94%
Removing identified outliers for customers...
See table for confirmation. 


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval


In [ ]:
# train = train.drop(columns=['Sales']), np.log1p(df_train['Sales'])
# valid = valid.drop(columns = ['Sales']), np.log1p(valid['Sales'])


In [ ]:
# imputing our data through the pipe class

processed_train_data = (train.
pipe(date_time).
pipe(to_category).
pipe(one_hot))

In [ ]:
processed_validation_data = (valid.
pipe(date_time).
pipe(to_category).
pipe(one_hot))

In [ ]:
processed_test_data = (test.
pipe(date_time).
pipe(to_category).
pipe(one_hot))

In [ ]:
processed_train_data.shape

(804056, 39)

In [ ]:
train = process_train_valid(train)
valid = process_train_valid(valid)
test = process_train_valid(test, isTest=True)

In [ ]:
train.shape

(804056, 21)

In [ ]:
valid.shape

(40282, 21)

In [ ]:
test.shape

(41088, 20)

In [ ]:
train.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth,Sales
284584,314,5,1,0,0,1,1,3560.0,10.0,2001.0,...,31.0,2013.0,2015,6,19,25,164.0,22.50,0,5032
417104,458,5,1,0,0,3,1,3390.0,0.0,0.0,...,0.0,0.0,2015,6,19,25,24186.0,24186.25,0,7362
875290,961,5,1,0,0,4,3,9430.0,0.0,0.0,...,0.0,0.0,2015,6,19,25,24186.0,24186.25,0,7764
119580,132,5,1,0,0,4,3,1040.0,0.0,0.0,...,27.0,2012.0,2015,6,19,25,24186.0,35.50,0,7964
807974,887,5,1,0,0,4,1,19700.0,0.0,0.0,...,37.0,2009.0,2015,6,19,25,24186.0,69.00,0,7776


In [ ]:
import pickle

from pickle import dump


In [ ]:
train_data_processed = train

with open('train_data_processed.pkl', 'wb') as file:
    pickle.dump(train_data_processed, file)

In [ ]:
test_data_processed = test

with open('test_data_processed.pkl', 'wb') as file:
    pickle.dump(test_data_processed, file)

In [ ]:
valid_data_processed = valid

with open('valid_data_processed.pkl', 'wb') as file:
    pickle.dump(valid_data_processed, file)

In [ ]:
with open('train_data_processed.pkl', 'rb') as train_file:

    train_data = pickle.load(train_file)

In [ ]:
train_data.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth,Sales
284584,314,5,1,0,0,1,1,3560.0,10.0,2001.0,...,31.0,2013.0,2015,6,19,25,164.0,22.50,0,5032
417104,458,5,1,0,0,3,1,3390.0,0.0,0.0,...,0.0,0.0,2015,6,19,25,24186.0,24186.25,0,7362
875290,961,5,1,0,0,4,3,9430.0,0.0,0.0,...,0.0,0.0,2015,6,19,25,24186.0,24186.25,0,7764
119580,132,5,1,0,0,4,3,1040.0,0.0,0.0,...,27.0,2012.0,2015,6,19,25,24186.0,35.50,0,7964
807974,887,5,1,0,0,4,1,19700.0,0.0,0.0,...,37.0,2009.0,2015,6,19,25,24186.0,69.00,0,7776


In [ ]:
# compression_out = dict(method='zip', archive_name = 'train_out.csv')

train_data_processed.to_csv('train_out.csv')


In [ ]:
test_data_processed.to_csv('test_out.csv')

In [ ]:
valid_data_processed.to_csv('valid_out.csv')

In [32]:
url = "https://raw.githubusercontent.com/Charliebond125/CE889_Group_Project/main/train_out.csv"

#https://github.com/Charliebond125/CE889_Group_Project/blob/7811c5490255fc0982b50d7fcfaa367908e01776/test_out.csv

test_train = pd.read_csv(url)

In [33]:
test_train.head()

# unamed: 0 = store ID. Data has been shuffled

,Unnamed: 0,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,...,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth,Sales
0,284584,314,5,1,0,0,1,1,3560.0,10.0,...,31.0,2013.0,2015,6,19,25,164.0,22.50,0,5032
1,417104,458,5,1,0,0,3,1,3390.0,0.0,...,0.0,0.0,2015,6,19,25,24186.0,24186.25,0,7362
2,875290,961,5,1,0,0,4,3,9430.0,0.0,...,0.0,0.0,2015,6,19,25,24186.0,24186.25,0,7764
3,119580,132,5,1,0,0,4,3,1040.0,0.0,...,27.0,2012.0,2015,6,19,25,24186.0,35.50,0,7964
4,807974,887,5,1,0,0,4,1,19700.0,0.0,...,37.0,2009.0,2015,6,19,25,24186.0,69.00,0,7776


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train)

StandardScaler()

In [39]:
scaler.transform(train)

array([[-0.75976979,  0.858232  ,  1.11013958, ..., -1.00414278,
        -0.4495676 , -0.65489785],
       [-0.31218881,  0.858232  ,  1.11013958, ...,  0.99735462,
        -0.4495676 ,  0.17193602],
       [ 1.25123643,  0.858232  ,  1.11013958, ...,  0.99735462,
        -0.4495676 ,  0.31459148],
       ...,
       [-0.88409784, -0.8820532 , -0.90078763, ..., -1.00600646,
         2.22435961, -1.339786  ],
       [-0.93072086, -0.8820532 , -0.90078763, ...,  0.9948697 ,
        -0.4495676 , -0.00939965],
       [-0.92139626, -0.8820532 , -0.90078763, ...,  0.9948697 ,
        -0.4495676 ,  2.75818722]])

In [41]:
scaler.mean_

array([5.58440345e+02, 3.52053215e+00, 4.47946412e-01, 1.48870228e-03,
       1.86729531e-01, 2.20488125e+00, 1.93681162e+00, 5.44503651e+03,
       4.92620663e+00, 1.36965231e+03, 4.98054862e-01, 1.15737561e+01,
       1.00196351e+03, 2.01377343e+03, 5.80050270e+00, 1.57105911e+01,
       2.34124116e+01, 7.73125688e+03, 1.21453511e+04, 1.68130080e-01,
       6.87748800e+03])

In [62]:
scaler.fit_transform(valid)

array([[-1.73126858,  0.86196377,  1.1882556 , ...,  1.02219536,
        -0.57568767, -0.5910642 ],
       [ 0.58727246,  0.86196377,  1.1882556 , ...,  1.02219536,
        -0.57568767,  1.36622469],
       [ 0.66497156,  0.86196377,  1.1882556 , ...,  1.02219536,
        -0.57568767, -0.60508335],
       ...,
       [-1.33034124,  1.44307883, -0.84156977, ..., -0.97643242,
        -0.57568767, -0.93075897],
       [ 1.49790588,  1.44307883, -0.84156977, ..., -0.98034688,
        -0.57568767,  0.04339216],
       [-0.75847589,  1.44307883, -0.84156977, ..., -0.97978767,
        -0.57568767, -1.38943677]])

In [64]:
scaler.fit_transform(test)

array([[-1.73259592,  0.01033678,  1.23544154, ..., -0.76633345,
         1.17667518, -0.38215815],
       [-1.73259592, -0.48582866,  1.23544154, ..., -0.76633345,
         1.17667518, -0.38215815],
       [-1.73259592, -0.98199411,  1.23544154, ..., -0.76633345,
         1.17667518, -0.38215815],
       ...,
       [ 1.74571275, -1.47815955,  1.23544154, ...,  1.30391053,
        -0.8500866 , -0.38215815],
       [ 1.74571275,  1.49883311, -0.80942721, ...,  1.30391053,
        -0.85010758, -0.38215815],
       [ 1.74571275,  1.00266767, -0.80942721, ...,  1.30391053,
        -0.85010758, -0.38215815]])

In [65]:
def add_missing_dummy_columns( d, columns ):
        missing_cols = set( columns ) - set( d.columns )
        for c in missing_cols:
            d[c] = 0

def fix_columns( d, columns ):  

    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols: print ("extra columns:", extra_cols)

    d = d[ columns ]
    return d

testFeatures= fix_columns(valid, train.columns)

In [67]:
def add_missing_columns( d, columns ):
        missing_cols = set( columns ) - set( d.columns )
        for c in missing_cols:
            d[c] = 0

def fix_columns( d, columns ):  

    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols: print ("extra columns:", extra_cols)

    d = d[ columns ]
    return d

testFeatures= fix_columns(test, train.columns)

In [66]:
print(train.shape)
print(valid.shape)

(804056, 21)
(40282, 21)


In [68]:
print(train.shape)
print(test.shape)

(804056, 21)
(41088, 21)


In [69]:
train_copy = train
test_copy = test
valid_copy = valid

In [70]:
test.columns.tolist()

['Store',
 'DayOfWeek',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'Year',
 'Month',
 'Day',
 'WeekOfYear',
 'CompetitionOpen',
 'PromoOpen',
 'IsPromoMonth',
 'Sales']

In [71]:
train.columns.tolist()

['Store',
 'DayOfWeek',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'Year',
 'Month',
 'Day',
 'WeekOfYear',
 'CompetitionOpen',
 'PromoOpen',
 'IsPromoMonth',
 'Sales']

In [72]:
valid.columns.tolist()

['Store',
 'DayOfWeek',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'Year',
 'Month',
 'Day',
 'WeekOfYear',
 'CompetitionOpen',
 'PromoOpen',
 'IsPromoMonth',
 'Sales']

In [ ]:
from sklearn.model_selection import train_test_split



In [80]:
target = ['Sales']

y_train = train[target]
y_test = valid[target]


In [87]:
features = [20]

In [97]:
X_train = train
X_test = train

In [95]:
train_copy = train.drop(['Sales'], axis=1)

In [96]:
train_copy.shape

(804056, 20)

In [81]:
y_train.head()

,Sales
284584,5032
417104,7362
875290,7764
119580,7964
807974,7776


In [82]:
y_test.head()

,Sales
0,5263
679364,10708
702362,5224
683890,7763
17714,9593
